In [1]:
import gym
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
from functools import reduce
from itertools import count
from gymenv.PlaytimeEnv import PlaytimeEnv
from gymenv.action_manager import ActionModel
from objects.Maneuver import Mission_Maneuver
from objects.AllManeuvers import LIST_MAN
from objects.Plane import ULM
from function.tools import *
from function.j_methods import * 



C:\Users\User\AppData\Roaming\Python\Python38\site-packages\seaborn\rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
c:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\setuptools\_distutils\version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


Cet avion peut voler pendant  52438  s, soit  873  min, soit 2039  km à une vitesse moyenne de  140  km/h et sa vitesse max sera de  200  km/h


The list of parameters that follow in the next cell represent the gameplan parameters.
Among them, there are :
- The **plane**, with a custom object, containing informations about the maximum and minimum speed, fuel consumption, fuel tank, max flight time...  
    - TODO : We need to complete those information in the futur, by adding the payload, having a custom-made more accurate fuel consumption function...
    
- The **goal distance**, representing the distance between the plane and the mission location.  
    - TODO : Add a time constraint to make a decision about how to travel this distance at the beginning of the mission (speed and altitude).  
    - TODO : Add a zone constraint with SAM or ennemies that will change the trajectory (instead of a direct line).  

- The **RtB distance** or **Return to base distance** : this is the distance between the mission location and the return point, an allied base, supply point or safe location.  
    - TODO : Use the time constraint about the synchroTime (following in the next parameters) to make a decision about how to travel this distance considering what has been done during the mission.  
    - TODO : Like the goal distance, add a zone constraint with SAM or ennemies that will change the trajectory.

- The amount of **fuel available** at the beginning of the mission. This amount of fuel can't be higher than what can the plane hold.
    - No Todos 
    
- The **meteo** during this mission. Depending of the weather, it has an impact on the altitude or the radius of action.  
    - TODO : We can add a changing weather in the environment to add a certain adaptation and randomness.

- The **type of the mission** : for now there are only two (or three with the **SWEEP** but NYI with the **CAS** and **SCAR** missions. The first one (*Close Air Support*) consists of helping and supporting allies on the ground with recon or attack maneuvers. The second one, *Strike Coordination And Recce*, is an intimidation and research on zone mission. Depending on the type of mission, some maneuvers are recommended / needed in order to complete the mission.

    - TODO : Implement new maneuvers or change parameters according to the mission : it would be good to have a real difference between the two actual types of mission.
    - TODO : Implement **SWEEP**, with new maneuvers and type of decision needed

- The **power relations** between the pilote / allies and their ennemies. It's important to know if the pilote has the advantage during the mission : if he has the advantage, then he is safer and can reduce his average speed in order to consume less. On the contrary, if he is at disadvantage, he will have to counter the balance by speeding up.

  - TODO : add a changing power relations, reducing as time passes (we assume the ennemies have been eliminated)
  - TODO : add more details, calculate it with the number of allies, opponents, weaponry, machinery, type of support...
  - TODO : add local power relation, changing according to the zone in which the maneuver will be done.

- The **minimum time** spent on the zone in order to accomplish the mission. This value is required by the support when they ask the pilote if the mission can be done. If it can be reached, it will either go to the synchronization time or consume all the fuel remaining. If not, then it will consume all the fuel to try and reach this minimum time.
    - No Todos

- The **synchronization time** is the time at which the pilote need to be at the supply point in order to get refueled at the right moment. It is the maximum time available on the mission, it is needed to reach the closest time. If there are no synchronization time, it will consume all the fuel remaining.
  - No Todos 

- TODO : Add objective points for the mission : each objective needs a certain maneuver and has a few local parameters like the type of action needed, the zone of action (for maneuvers like **Spiral** or **ZigZag**), the power balance, etc. Plus, it can help in the choice of the number and the different maneuvers, instead of having an unknown amount of maneuvers.

- TODO : Add a path solver between the objective points. After a maneuver, we need the pilote to move to the next objective in the best way possible. This way, we can make the environment in 2 dimensions, considering the altitude and the distance. Moreover, we need to consider the potentiel threats between the objective points.


- TODO : Adapt gameplans parameters if the mission is a **SWEEP** : the global philosophy of this mission is completely different from the two others, maybe a whole new environment is needed.

- TODO : Add field data like topography. It can help in the path solving for the altitude among others.

In [1]:
param_list = {
    'Plane': [ULM],
    'GoalDistance': [0, 10, 15, 20, 30],
    'RtBDistance': [0, 10, 15, 20, 30],
    'FuelAvailable': [15, 20, 30, 35, 40, 60],
    # Start with just 3 states : sunny, cloudy or misty.
    'Meteo': ["Sunny", "Cloudy", "Misty"],
    'MissionType': [Mission_Maneuver.SCAR],  # , Mission_Maneuver.CAS
    # Add ennemies number afterwards, weaponry
    'Strength': ['Weak', 'Equal', 'Strong'],
    'TimeMin': [0, 300, 600, 1800],
    'SynchroTime': [0, 1800, 3600],
}
combinations = get_all_combinations(param_list)

NameError: name 'ULM' is not defined

In [3]:
import gym

from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.env_checker import check_env
# Parallel environments
env = PlaytimeEnv(combinations)

model = A2C("MultiInputPolicy", env, verbose=1)
model.learn(total_timesteps=10000)
model.save("a2c_playtime")

del model # remove to demonstrate saving and loading


c:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\utils\tensorboard\__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Fuel, Tmin, Tsync :  60 300 1800
Fuel, Tmin, Tsync :  20 0 0
Too much fuel consumed
Fuel, Tmin, Tsync :  15 600 0
Too much fuel consumed
Fuel, Tmin, Tsync :  30 300 3600
Fuel, Tmin, Tsync :  40 0 1800
Fuel, Tmin, Tsync :  15 300 1800
Too much fuel consumed
Fuel, Tmin, Tsync :  15 600 0
Too much fuel consumed
Fuel, Tmin, Tsync :  40 1800 3600
Fuel, Tmin, Tsync :  15 1800 1800
Fuel, Tmin, Tsync :  30 600 0
Too much fuel consumed
Fuel, Tmin, Tsync :  30 600 3600
Too much fuel consumed
Fuel, Tmin, Tsync :  60 300 3600
Fuel, Tmin, Tsync :  40 300 3600
Fuel, Tmin, Tsync :  30 0 0
Too much fuel consumed
Fuel, Tmin, Tsync :  60 0 3600
Fuel, Tmin, Tsync :  35 0 1800
Fuel, Tmin, Tsync :  30 1800 0
Too much fuel consumed
Fuel, Tmin, Tsync :  35 0 0
Too much fuel consumed
Fuel, Tmin, Tsync :  30 300 0
Too much fuel consumed
Fuel, Tmin, Tsync :  60 300 0
Too much fuel consumed
Fuel, Tmin, Tsync :  30 300 1

In [4]:

model = A2C.load("a2c_playtime")
nb_episode = 1000
nb_occ = 0
obs = env.reset(verbose=0)

while nb_occ != nb_episode:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    if dones == True:
        print("Reward : ", rewards)
        print([(m.name, m.meanspeed_kmh, m.altitude, m.distance) for m in env.maneuver_list])
        obs = env.reset(verbose=0)
        nb_occ += 1

Reward :  -99
[(<Maneuver_Mission.Zigzag: 4>, 112, 12682, 261.8407044966673)]
Too much fuel consumed
Reward :  -95250
[(<Maneuver_Mission.Zigzag: 4>, 184, 3382, 339.84070449666723)]
Reward :  339
[(<Maneuver_Mission.Wheel: 1>, 194, 1470, 14.566370614359172), (<Maneuver_Mission.Spiral: 3>, 101, 18715, 85.8089636779981), (<Maneuver_Mission.Wheel: 1>, 105, 23318, 20.84955592153876)]
Too much fuel consumed
Reward :  -89423
[(<Maneuver_Mission.Wheel: 1>, 146, 23532, 20.84955592153876), (<Maneuver_Mission.Wheel: 1>, 105, 3138, 20.84955592153876), (<Maneuver_Mission.Wheel: 1>, 105, 7979, 20.84955592153876), (<Maneuver_Mission.Wheel: 1>, 114, 8047, 20.84955592153876), (<Maneuver_Mission.Wheel: 1>, 105, 21984, 20.84955592153876), (<Maneuver_Mission.Wheel: 1>, 182, 21236, 20.84955592153876), (<Maneuver_Mission.Wheel: 1>, 114, 14132, 20.84955592153876), (<Maneuver_Mission.Wheel: 1>, 105, 13640, 14.566370614359172), (<Maneuver_Mission.Wheel: 1>, 105, 20801, 20.84955592153876), (<Maneuver_Mission.W